In [ ]:
from random import randint
from collections import deque


STRIKE_COUNT = 10
STRIKE = "strike"
SPARE = "spare"
NORMAL = "normal"


down_list = [10, 
             10, 
             9, 1, 
             5, 2
             ]

idx = 0

class Frame():
    def __init__(self, num: int):
        self.is_strike = False
        self.is_spare = False
        self.score = 0
        self.score_acc_done = True if num == 0 else False
        self.throwing_count = 0  # 최대 2
        self.strike_score_count = 2
        self.spare_score_count = 1
        self.num: int = num
    
    
    def __str__(self):
        return f"num({self.num}), score({self.score}), is_strike({self.is_strike}), is_spare({self.is_spare})"
    
    
    def is_complete(self) -> bool:
        if self.is_strike and self.throwing_count == 1 and self.strike_score_count == 0:
            return True
        if self.is_spare and self.throwing_count == 2 and self.spare_score_count == 0:
            return True
        if self.throwing_count == 2:
            return True
        
        return False
    
    
    def process_throwing_result(self, downed_pin_count: int) -> str:
        if self.throwing_count >= 2:
            raise Exception("unreachable")

        if self.throwing_count == 0:
            self.score += downed_pin_count
            self.throwing_count += 1
            
            if downed_pin_count == STRIKE_COUNT:
                self.is_strike = True
                return STRIKE
            else:
                return NORMAL
            
        else:
            self.score += downed_pin_count
            self.throwing_count += 1
            
            if self.score == STRIKE_COUNT:
                self.is_spare = True
                return SPARE
            else:
                return NORMAL
        
    
    def add_additional_score(self, downed_pin_count: int) -> bool:
        if self.is_strike:
            return self.add_strike_score(downed_pin_count)
        else:
            return self.add_spare_score(downed_pin_count)
    
            
    
    def add_strike_score(self, downed_pin_count: int) -> bool:
        if self.strike_score_count <= 0 or not self.is_strike:
            raise Exception("unreachable")
        
        self.score += downed_pin_count
        self.strike_score_count -= 1
        return self.strike_score_count == 0
        
    
    def add_spare_score(self, downed_pin_count: int) -> bool:
        if self.spare_score_count <= 0 or not self.is_spare:
            raise Exception("unreachable")
        
        self.score += downed_pin_count
        self.spare_score_count -= 1
        return self.spare_score_count == 0


class Balling():
    def __init__(self, force_strike=False):
        self.complete_frames: list[Frame] = []
        self.not_complete_frames: deque[Frame] = deque()
        self.force_strike = force_strike
        self.frame_num = 0
    
    
    def throw_a_ball(self, remaining_pin_count=10, force_strike=False) -> int:
        """
        쓰러뜨린 핀의 개수를 반환한다.
        """
        
        global idx
        global down_list
        result = down_list[idx]
        idx += 1
        print("\t쓰러진 핀 개수:", result)
        return result
        
        #if force_strike: return 10
        
        #return randint(0, remaining_pin_count)
    
    
    def acc_strike_and_spare(self, downed_pin_count: int):
        for _ in range(len(self.not_complete_frames)):
            f = self.not_complete_frames.popleft()
            
            complete = f.add_additional_score(downed_pin_count)
            if f.num == 1:
                print("="*50)
                print(f)
                print(complete)
                print("="*50)
            if complete:
                self.complete_frames.append(f)
            else:
                self.not_complete_frames.append(f)
        
        self.complete_frames = sorted(self.complete_frames, key=lambda f: f.num)
        self.not_complete_frames = deque(sorted(self.not_complete_frames, key=lambda f: f.num))
    
    
    def acc_scores(self):
        for f in self.complete_frames:
            if f.num == 1:
                print(f"\t[acc_scores] frame.num == 1, f.score_acc_done({f.score_acc_done})")
                for f in self.complete_frames:
                    print("\tcomplete) ", f)
                
            if not f.score_acc_done:
                target = int(f.num - 1)
                find = [frm for frm in self.complete_frames if frm.num == target]
                
                if len(find) == 0:
                    if f.num == 1:
                        print("못 찾았다.")
                    continue
                
                prev_f = find[0]
                if prev_f.score_acc_done:
                    f.score += prev_f.score
                    f.score_acc_done = True
    
    
    def play_a_frame(self):
        f = Frame(self.frame_num)
        self.frame_num += 1
        
        remaining_pin_count = STRIKE_COUNT
        downed_pin_count = self.throw_a_ball(remaining_pin_count, self.force_strike)
        self.acc_strike_and_spare(downed_pin_count)
        remaining_pin_count -= downed_pin_count
        
        result = f.process_throwing_result(downed_pin_count)
        print("\tthrowing result:", result)
        if result == STRIKE:
            self.not_complete_frames.append(f)
            
        else:
            downed_pin_count = self.throw_a_ball(remaining_pin_count, self.force_strike)
            self.acc_strike_and_spare(downed_pin_count)
            result = f.process_throwing_result(downed_pin_count)
            
            print("\tthrowing result:", result)
            
            if result == SPARE:
                self.not_complete_frames.append(f)
            else:
                self.complete_frames.append(f)
        
        self.acc_scores()
        
    
    
    def show_scores(self):
        self.complete_frames = sorted(self.complete_frames, key=lambda x: x.num)
        scores = [s.score for s in self.complete_frames]
        print(scores)
    
    def dump_frames(self):
        for f in self.complete_frames:
            print("\tcomplete) ", f)
        for f in self.not_complete_frames:
            print("\tnot complete) ",f)
        print("dump done!")


b = Balling()
b.play_a_frame()
b.dump_frames()

b.play_a_frame()
b.dump_frames()

b.play_a_frame()
b.dump_frames()

b.play_a_frame()
b.dump_frames()

b.show_scores()
        

        
